In [12]:
import numpy as np
from simulation.generate_path import generate_path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from estimation.distance_sensor import distance_sensors
from estimation.iterative_estimator import iterative_estimator
from estimation.kalman_filter_from_points import kalman_filter_from_points
from estimation.kalman_filter_from_points_with_acc_mahal import kalman_filter_from_points_acc
from estimation.improved_kalman_filter_from_points_with_acc import improved_kalman_filter_from_points_acc

In [13]:
mahal = []
NN = []
iterative = []

mahal_outliers = []
NN_outliers = []
iterative_outliers = []

mahal_non_outliers = []
NN_non_outliers = []
iterative_non_outliers = []

mahal_straight_outliers = []
NN_straight_outliers = []
iterative_straight_outliers = []
             
mahal_turn_outliers = []
NN_turn_outliers = []
iterative_turn_outliers = []
count = 0

for i in np.arange(100):
    seed = np.random.randint(0, 100000)
    np.random.seed(seed)
    # create the path
    target_initial_pos = np.array([0, 0, 5000])
    target_speed_xy = 50
    target_speed_z = 10
    target_rot_speed = 1
    time_res = 0.1
    sensor_sigma = 15
    
    path1 = generate_path(0, target_speed_xy, target_speed_z, target_initial_pos, time_res)
    path1.add_straight_interval(100)
    path1.add_xy_turn_interval(90, -np.deg2rad(target_rot_speed))
    path1.add_straight_interval(100)
    
    outliers = np.random.randint(0, 100, size=len(path1.path))
    outliers = outliers > (100 - 2)

    # create noisy sensors
    sensors = distance_sensors([[-5000,0,0], [400, -7400, 0], [800, 800, 0]], sensor_sigma)
    sensors.calculate_measurements(path1.path)
    
    sensors_noisy = distance_sensors([[-5000,0,0],[ 400, -7400, 0],[800, 800, 0]], 200)
    sensors_noisy.calculate_measurements(path1.path)
    
    # estimate the path
    it_est = iterative_estimator(sensors, path1.path[0,:])
    estimated_path = it_est.estimate_path()
    cov_mat_it = it_est.get_cov_err_for_kf(estimated_path)
    
    it_est_noisy = iterative_estimator(sensors_noisy, path1.path[0,:])
    estimated_path_noisy = it_est_noisy.estimate_path()
    
    outliers[:50] = 0
    # create path with outliers
    for i in np.arange(len(outliers)):
        if outliers[i]:    
            #estimated_path_noisy[i,:] = estimated_path[i,:]
            estimated_path[i,:] = estimated_path_noisy[i,:]
            
    # run KF on path
    sigma_a = 1
    sigma_v = 100
    
    improved_KF = improved_kalman_filter_from_points_acc(time_res, sigma_a, sigma_v, non_diag_reduction_ratio=2)
    kf_acc = kalman_filter_from_points_acc(time_res, sigma_a, sigma_v, non_diag_reduction_ratio=2)
    
    kf_path_acc = kf_acc.filter_path(estimated_path, cov_mat_it)
    kf_path_acc_noisy = improved_KF.filter_path(estimated_path)
    
    it_est_MSE = np.sqrt(np.sum(np.square(estimated_path-path1.path), 1))
    kf_acc_MSE = np.sqrt(np.sum(np.square(kf_path_acc-path1.path), 1))
    improved_kf_acc_MSE = np.sqrt(np.sum(np.square(kf_path_acc_noisy-path1.path), 1))
    
    if ((np.mean(kf_acc_MSE)<1000) & (np.mean(improved_kf_acc_MSE)<1000)):
        mahal = np.append(mahal, kf_acc_MSE)
        NN = np.append(NN, improved_kf_acc_MSE)
        iterative = np.append(iterative, it_est_MSE)
        count = count + 1
        
        mahal_outliers = np.append(mahal_outliers, kf_acc_MSE[outliers])
        NN_outliers = np.append(NN_outliers, improved_kf_acc_MSE[outliers])
        iterative_outliers = np.append(iterative_outliers, it_est_MSE[outliers])
        
        mahal_non_outliers = np.append(mahal_non_outliers, kf_acc_MSE[1-outliers])
        NN_non_outliers = np.append(NN_non_outliers, improved_kf_acc_MSE[1-outliers])
        iterative_non_outliers = np.append(iterative_non_outliers, it_est_MSE[1-outliers])
        
        mahal_straight_outliers = np.append(mahal_straight_outliers, kf_acc_MSE[1000:1898])
        NN_straight_outliers = np.append(NN_straight_outliers, improved_kf_acc_MSE[1000:1898])
        iterative_straight_outliers = np.append(iterative_straight_outliers, it_est_MSE[1000:1898])
    
        mahal_turn_outliers = np.append(mahal_turn_outliers, (kf_acc_MSE[:1000], kf_acc_MSE[1898:]))
        NN_turn_outliers = np.append(NN_turn_outliers, (improved_kf_acc_MSE[:1000], improved_kf_acc_MSE[1898:]))
        iterative_turn_outliers = np.append(iterative_turn_outliers, (it_est_MSE[:1000], it_est_MSE[1898:]))
    
    
mahal = np.mean(mahal)
NN = np.mean(NN)
iterative = np.mean(iterative)

print(count)
print("mahal: " +str(mahal))
print("NN: " + str(NN))
print("iterative: " + str(iterative))
print("\n")

print("mahal_outliers: " + str(np.mean(mahal_outliers)))
print("NN_outliers: " + str(np.mean(NN_outliers)))
print("iterative_outliers: " + str(np.mean(iterative_outliers)))
print("\n")

print("mahal_non_outliers: " + str(np.mean(mahal_non_outliers)))
print("NN_non_outliers: " + str(np.mean(NN_non_outliers)))
print("iterative_non_outliers: " + str(np.mean(iterative_non_outliers)))
print("\n")

print("mahal_straight_outliers: " + str(np.mean(mahal_straight_outliers)))
print("NN_straight_outliers: " + str(np.mean(NN_straight_outliers)))
print("iterative_straight_outliers: " + str(np.mean(iterative_straight_outliers)))
print("\n")

print("mahal_turn_outliers: " + str(np.mean(mahal_turn_outliers)))
print("NN_turn_outliers: " + str(np.mean(NN_turn_outliers)))
print("iterative_turn_outliers: " + str(np.mean(iterative_turn_outliers)))

100
mahal: 22.79063306074254
NN: 22.217971298385002
iterative: 67.2308500123678


mahal_outliers: 24.73522973363382
NN_outliers: 38.20609575985075
iterative_outliers: 818.2988552660082


mahal_non_outliers: 18.81335654683056
NN_non_outliers: 23.82510263150761
iterative_non_outliers: 28.9682438321739


mahal_straight_outliers: 23.174551431914146
NN_straight_outliers: 22.517845346018326
iterative_straight_outliers: 70.0446656059822


mahal_turn_outliers: 22.618253712086496
NN_turn_outliers: 22.083327850997648
iterative_turn_outliers: 65.96744681083493
